
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_2333724/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-26 01:41:45.881391: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 01:41:45.915930: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 01:41:45.915977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 01:41:45.917304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 01:41:45.923322: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 01:41:45.924480: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[15,7], [15,8]],
            regularizers=[ 0.1,0.02],
            learning_rate=[ 0.1,0.02])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.1, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  1.3440682479995303
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.044023733000358334
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.07440357899940864
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.032943893000265234
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.03805562900015502
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.031222614999933285
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.032746650999797566
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_0_9,0.800218,0.824926,-0.680061,0.423110,0.221099,0.195924,0.993830,0.571409
model_2_0_7,0.808430,0.822284,-0.523404,0.465558,0.212012,0.198880,0.901160,0.529365
model_2_0_8,0.812790,0.827260,-0.502575,0.474388,0.207186,0.193312,0.888839,0.520619
model_2_0_0,0.818305,0.834136,-0.019312,0.614318,0.201083,0.185617,0.602968,0.382018
model_2_0_1,0.819236,0.800512,-0.021360,0.593631,0.200052,0.223245,0.604179,0.402509
model_2_0_2,0.847190,0.850058,0.117715,0.662353,0.169116,0.167798,0.521910,0.334439
model_2_0_4,0.848487,0.840775,0.080526,0.646348,0.167681,0.178187,0.543909,0.350292
model_2_0_3,0.848553,0.849898,0.120718,0.663101,0.167607,0.167978,0.520134,0.333698
model_2_0_6,0.850319,0.835248,0.042106,0.632245,0.165652,0.184373,0.566636,0.364262
model_2_0_5,0.851673,0.841388,0.088737,0.649022,0.164155,0.177502,0.539052,0.347643


DataFrame salvo em ../results/metrics_2_0
+++++++++++ [2_1] | 1 ++++++++++++++++++
Elapsed time:  1.0843793840003855
Stopped at epoch:  15
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_1] | 2 ++++++++++++++++++
Elapsed time:  0.3446979939999437
Stopped at epoch:  6
1/1 [==============================] - 0s 29ms/step
+++++++++++ [2_1] | 3 ++++++++++++++++++
Elapsed time:  0.44920022600035736
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [2_1] | 4 ++++++++++++++++++
Elapsed time:  0.7046088459992461
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_1] | 5 ++++++++++++++++++
Elapsed time:  0.6890850099998715
Stopped at epoch:  8
1/1 [==============================] - 0s 20ms/step
+++++++++++ [2_1] | 6 ++++++++++++++++++
Elapsed time:  0.5066544629999044
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [2_1] | 7 ++++++++++++++++++
Elapsed time:  0.7620870420

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_1_0,0.821798,0.848011,0.788643,0.819959,0.197217,0.161751,0.227485,0.192685
model_2_1_1,0.825461,0.847388,0.790067,0.820306,0.193164,0.162413,0.225953,0.192314
model_2_1_2,0.836222,0.915293,0.768976,0.846072,0.181254,0.090147,0.248654,0.164739
model_2_1_3,0.846723,0.902661,0.781110,0.845165,0.169632,0.103590,0.235594,0.165709
model_2_1_4,0.852584,0.890075,0.795843,0.845512,0.163146,0.116985,0.219736,0.165338
model_2_1_5,0.854649,0.898361,0.788508,0.846402,0.160861,0.108167,0.227631,0.164386
model_2_1_6,0.876452,0.908150,0.787925,0.851280,0.136731,0.097749,0.228258,0.159165
model_2_1_7,0.885725,0.919784,0.802866,0.864475,0.126469,0.085368,0.212178,0.145043
model_2_1_8,0.886240,0.907033,0.816570,0.864248,0.125898,0.098938,0.197428,0.145286
model_2_1_24,0.890600,0.824637,0.738216,0.783790,0.121074,0.186625,0.281761,0.231395


...

DataFrame salvo em ../results/metrics_8_8
+++++++++++ [8_9] | 1 ++++++++++++++++++
Elapsed time:  0.44063699199978146
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [8_9] | 2 ++++++++++++++++++
Elapsed time:  0.4873466989993176
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [8_9] | 3 ++++++++++++++++++
Elapsed time:  0.6897314969992294
Stopped at epoch:  11
1/1 [==============================] - 0s 18ms/step
+++++++++++ [8_9] | 4 ++++++++++++++++++
Elapsed time:  0.36804934199972195
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [8_9] | 5 ++++++++++++++++++
Elapsed time:  0.6027456809988507
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [8_9] | 6 ++++++++++++++++++
Elapsed time:  0.502324999000848
Stopped at epoch:  7
1/1 [==============================] - 0s 18ms/step
+++++++++++ [8_9] | 7 ++++++++++++++++++
Elapsed time:  0.5054233459

2024-03-26 02:07:32.486484: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.8364375760011171
Stopped at epoch:  12
1/1 [==============================] - 0s 28ms/step
+++++++++++ [8_9] | 23 ++++++++++++++++++
Elapsed time:  0.3848578550005186
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [8_9] | 24 ++++++++++++++++++
Elapsed time:  0.6794027400010236
Stopped at epoch:  11
1/1 [==============================] - 0s 15ms/step
+++++++++++ [8_9] | 25 ++++++++++++++++++
Elapsed time:  0.7267032979998476
Stopped at epoch:  11
1/1 [==============================] - 0s 30ms/step


,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_8_9_0,0.852,0.810993,0.838116,0.840298,0.163793,0.187798,0.390198,0.283045


DataFrame salvo em ../results/metrics_8_9
